In [22]:
pip install pytorch-lightning

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [23]:
import torch
import torch.nn.functional as f
import torchvision
import torchvision.datasets as datasets
from torchvision.transforms import transforms
from torch import nn,optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split


import pytorch_lightning as pl

In [47]:
class lightning(pl.LightningModule):
  def __init__(self,input_size,num_classes):
    super().__init__()
    self.fc1=nn.Linear(input_size,50)
    self.fc2=nn.Linear(50, num_classes)
    self.loss=nn.CrossEntropyLoss()

  def forward(self,x):
    x=f.relu(self.fc1(x))
    x=self.fc2(x)

    return x

  def common_step(self,batch,batch_idx):
    x,y=batch
    x=x.reshape(x.size(0),-1)
    scores=self.forward(x)
    loss=f.cross_entropy(scores,y)
    return loss

  def training_step(self,batch,batch_idx):

    loss,scores=self.common_step(batch,batch_idx)
    self.log("train loss:",loss)
    return loss


  def validation_step(self,batch,batch_idx):
    loss,scores=self.common_step(batch,batch_idx)
    self.log("val loss:",loss)
    return loss

  def test_step(self,batch,batch_idx):
    loss,scores=self.common_step(batch,batch_idx)
    self.log("test loss:",loss)
    return loss

  def predict_step(self,batch,batch_idx):
    x,y=batch
    x=x.reshape(x.size(0),-1)
    scores=self.forward(x)
    preds=torch.argmax(scores,dim=1)
    return preds

  def configure_optimizers(self):
    return optim.Adam(self.parameters(), lr=0.01)

In [32]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [33]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3


In [34]:
entire_dataset=datasets.MNIST(root="dir",
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

train_ds,val_ds=random_split(entire_dataset, [50000, 10000])

test_ds = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)


In [35]:
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)

In [36]:
model_0=lightning(input_size=input_size,num_classes=num_classes).to(device)


In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_0.parameters(), lr=learning_rate)





In [48]:
trainer=pl.Trainer(accelerator="auto",
                   devices=1,
                   min_epochs=1,
                   max_epochs=3,
                   precision=16)

trainer.fit(model_0,train_loader,val_loader)
trainer.validate(model_0,val_loader)
trainer.test(model_0,test_loader)

INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


MisconfigurationException: No `configure_optimizers()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.

In [16]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # Forward
        scores = model_0(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

100%|██████████| 782/782 [00:08<00:00, 91.47it/s] 


In [20]:
model_0.to(device)
print(f"Accuracy on training set: {check_accuracy(train_loader, model_0)*100:.2f}")
print(f"Accuracy on validation set: {check_accuracy(val_loader, model_0)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model_0)*100:.2f}")

Accuracy on training set: 10.98
Accuracy on validation set: 10.98
Accuracy on test set: 11.08
